# Homework 11 - Generative Models

In this homework you will implement a simple generative adversarial network to generate new samples for the MNIST dataset. 

On the technical side you will learn about TensorBoard and how to use it to store your metrics and also the stored images.

In [0]:
%load_ext tensorboard
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime

TensorFlow 2.x selected.


In [0]:
# Load the MNIST dataset. Training data is enough. Normalize to [-1,1]. Add channel dimension of depth 1.
# Format to float32.

(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [0]:
# Define the generator:
# Dense layer: 7*7*64 neurons + BatchNorm + LeakyReLU
# Reshape to (batch size, 7, 7, 64)
# Transpose Convolutional Layer: 32 kernels of size (5,5) with strides (1,1) + "same" padding + BatchNorm + LeakyReLU
# Transpose Convolutional Layer: 16 kernels of size (5,5) with strides (2,2) + "same" padding + BatchNorm + LeakyReLU
# Transpose Convolutional Layer: 1 kernels of size (5,5) with strides (2,2) + "same" padding + TanH
# All layers in the generator don't use biases! Set parameter use_bias=False.
# NOTE: Go back to lecture 7 to make sure you use BatchNorm in combination with an 
# activation function in the correct way.
from tensorflow.keras.layers import Layer


def leakyRelu(x):
    return tf.nn.leaky_relu(x, alpha=0.01)

def tanH(x):
    return tf.nn.tanh(x)


class Generator(Layer):

  def __init__(self):
    
    super(Model, self).__init__() #init all

    self.dense = tf.keras.layers.Dense(units=7*7*64,
                                       activation=None,
                                       use_bias=False,
                                       input_shape=(100,)
                                       )
    self.convT1 = tf.keras.layers.Conv2DTranspose(filters=32,
                                                 kernel_size=(5,5),
                                                 strides=(1,1),
                                                 padding='same',
                                                 activation=None,
                                                 use_bias=False
                                                 )
    self.convT2 = tf.keras.layers.Conv2DTranspose(filters=16,
                                                 kernel_size=(5,5),
                                                 strides=(2,2),
                                                 padding='same',
                                                 activation=None,
                                                 use_bias=False
                                                 )
    self.convT3 = tf.keras.layers.Conv2DTranspose(filters=1,
                                                  kernel_size=(5,5),
                                                  strides=(2,2),
                                                  padding='same',
                                                  activation=None,
                                                  use_bias=False
                                                  )
    self.batch_norm = tf.keras.layers.BatchNormalization()

  def call(self, x, is_training):
    x = self.dense(x)
    x = self.batch_norm(x, training=is_training)
    x = leakyRelu(x)
    x = self.convT1(x)
    x = self.batch_norm(x, training=is_training)
    x = leakyRelu(x)
    x = self.convT2(x)
    x = self.batch_norm(x, training=is_training)
    x = leakyRelu(x)
    x = self.convT3(x)
    x = tanH(x)

    return x

                                       



In [0]:
# Define the discriminator:
# Conv layer: 8 kernels of size (5,5) with strides (2,2) + "same" padding + LeakyReLU
# Conv layer: 16 kernels of size (5,5) with strides (2,2) + "same" padding + LeakyReLU
# Flatten
# Dense layer: 1 unit, Sigmoid activation

class Discriminator(Layer):

  def __init__(self):

    super(Model, self).__init__()

    self.conv1 = tf.keras.layers.Conv2D(filters=8,
                                        kernel_size=(5,5),
                                        strides=(2,2),
                                        padding='same',
                                        activation=leakyRelu
                                        )
    self.conv2 = tf.keras.layers.Conv2D(filters=16,
                                        kernel_size=(5,5),
                                        strides=(2,2),
                                        padding='same',
                                        activation=leakyRelu
                                        )
    self.flatten = tf.keras.layers.Flatten()
    self.dense =tf.keras.layers.Dense(units=1,
                                      activation='sigmoid'
                                      )
  
  def call(self,x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.flatten(x)
    x = self.dense(x)

    return x

In [0]:
# Define the dataset. Shuffle.
# We will train with batches of 64 (32 real images, 32 fake images).
# So batch in 32.
### YOUR CODE HERE ###

########################

In [0]:
# Define the losses. It makes sense to first program the training loop and then come back here!

# Define the loss for the generator.
def generator_loss(probabilities):
    ### YOUR CODE HERE ###
    # Get only the output probabilities for the fake images.
    
    # Create the label vector indicating that the images are correct (=1).

    # Use binary cross entropy loss to compute the loss. tf.keras.losses.BinaryCrossentropy()

    ########################
    return loss

# Define the loss for the discriminator.
def discriminator_loss(probabilities):
    ### YOUR CODE HERE ###
    # Create the label vector indicating which images are real and which are fake.

    # Use binary cross entropy loss to compute the loss.

    ########################
    return loss

In [0]:
!rm -rf ./logs/ 
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

# For me the in-notebook tensorboard sometimes doesn't work. In this case maybe just use your terminal
# if you work on your own machine. (comment out the following line)
%tensorboard --logdir logs/

tf.keras.backend.clear_session()

# Initialzing generator and discriminator.
generator = Generator()
discriminator = Discriminator()
# Define size of latent variable vector.
z_dim = 50
# Define random seed to use for generating 8 images for supervision.
seed = tf.random.normal(shape=[8,z_dim])

# Initialize two optimizers (one for generator, one for discriminator).
# During training you will have to do every step twice (computing loss, computing gradients,
# applying gradients, storing loss in summary). Namely once for generator and once for discriminator.
gen_optimizer = tf.keras.optimizers.Adam(1e-4)
dis_optimizer = tf.keras.optimizers.Adam(1e-4)

step=0

for epochs in range(6):
    for real in dataset:
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
            
            ### YOUR CODE HERE ###
            # Generate random noise vector. tf.random.normal()

            # Generate fake images with generator.

            # Merge fake and real images to a long vector. tf.concat()

            # Compute output from discriminator.
            
            # Compute loss, compute gradients, apply gradients, store summaries.

            ########################
            
        # Every 100 steps generate images from the defined seed and 
        # store to supervise how well the generator works.
        if step % 100 == 0:
            ### YOUR CODE HERE ###

            ########################
            
        step += 1